In [ ]:
!pip install transformers accelerate bitsandbytes pillow camelot-py[cv]
!pip install langchain chromadb sentence-transformers  pillow gradio
!pip install -qU langchain-text-splitters
!pip install langchain-chroma
!pip install langchain-huggingface
!pip install PyMuPDF
!pip install ftfy regex tqdm
!pip install  pandas faiss-cpu
!apt-get install ghostscript python3-tk -y
!pip install optimum

In [ ]:
import fitz
import os

PAGE_IMG_DIR = "/content/page_images"
os.makedirs(PAGE_IMG_DIR, exist_ok=True)

def render_pages(pdf_path, dpi=300):
    doc = fitz.open(pdf_path)
    zoom = dpi / 72
    mat = fitz.Matrix(zoom, zoom)

    pages = []
    for i in range(len(doc)):
        pix = doc[i].get_pixmap(matrix=mat, alpha=False)
        path = f"{PAGE_IMG_DIR}/page_{i+1}.png"
        pix.save(path)
        pages.append({"page": i+1, "path": path})
    return pages


In [ ]:
import cv2
import numpy as np

CROP_DIR = "/content/diagram_crops"
os.makedirs(CROP_DIR, exist_ok=True)

def extract_diagrams_from_page(page_img_path, page, source):
    img = cv2.imread(page_img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # --- binarize ---
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(
        blur, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 15, 3
    )

    # --- STRONG closing to connect diagram parts ---
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (35, 35))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    # --- find big connected regions ---
    contours, _ = cv2.findContours(
        closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    crops = []
    idx = 0

    for c in contours:
        x, y, w, h = cv2.boundingRect(c)

        if w * h > 80000:   # only BIG diagram areas
            pad = 15
            crop = img[
                max(0,y-pad):min(img.shape[0], y+h+pad),
                max(0,x-pad):min(img.shape[1], x+w+pad)
            ]

            out_path = f"{CROP_DIR}/{os.path.basename(source)}_p{page}_fig{idx}.png"
            cv2.imwrite(out_path, crop)

            crops.append({
                "page": page,
                "source": source,
                "path": out_path
            })
            idx += 1

    return crops


In [ ]:
pdf_path = "Enter PDF path"

pages = render_pages(pdf_path)

diagram_db = []

for p in pages:
    crops = extract_diagrams_from_page(p["path"], p["page"], pdf_path)
    for c in crops:
        diagram_db.append({
            "page": p["page"],
            "path": c
        })

print("Total diagrams:", len(diagram_db))


In [ ]:
# ============================================================
# IMPORTS
# ============================================================
import fitz
import numpy as np
import pandas as pd
import gradio as gr
import torch
import camelot
import gc
import re
from pathlib import Path
from PIL import Image
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Blip2Processor,
    Blip2ForConditionalGeneration
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from typing import List, Tuple

# ============================================================
# CONFIG
# ============================================================
PDF_PATH = "PDF Path"
IMAGES_FOLDER = Path("/content//diagram_crops")
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
BLIP2_MODEL = "Salesforce/blip2-opt-2.7b"

PAGE_NUM_PATTERN = re.compile(r"_p(\d+)_", re.IGNORECASE)
IMG_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".webp"}
MAX_IMG_SIZE = 1024

# ============================================================
# TEXT EXTRACTION WITH LANGCHAIN
# ============================================================
def extract_text_with_langchain(pdf_path):
    """Extract text from PDF and split into chunks using LangChain"""
    doc = fitz.open(pdf_path)
    full_text = ""
    page_text_map = {}

    for i in range(len(doc)):
        txt = doc[i].get_text("text").strip()
        if txt:
            full_text += f"\n\n--- Page {i + 1} ---\n\n{txt}"
            page_text_map[i + 1] = txt

    # LangChain text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks = text_splitter.split_text(full_text)

    # Create LangChain documents
    documents = []
    for i, chunk in enumerate(chunks):
        # Extract page number from chunk
        page_match = re.search(r"--- Page (\d+) ---", chunk)
        page_num = int(page_match.group(1)) if page_match else None

        documents.append(
            Document(
                page_content=chunk,
                metadata={"source": "pdf", "chunk": i, "page": page_num}
            )
        )

    return documents, page_text_map

# ============================================================
# TABLE EXTRACTION - FIXED FOR NA VALUES
# ============================================================
def extract_tables(pdf_path):
    """Extract tables with rich context"""
    tables = camelot.read_pdf(
        pdf_path,
        pages="all",
        flavor="lattice",
        strip_text="\n"
    )

    extracted = []
    table_documents = []

    for i, table in enumerate(tables):
        df = table.df.copy()

        # CRITICAL FIX: Replace pandas NA with empty string BEFORE any operations
        df = df.fillna('')

        # Clean up DataFrame - remove completely empty rows and columns
        df = df.replace('', np.nan).dropna(how='all').dropna(axis=1, how='all').fillna('')

        # Get column names from first row
        col_names = " | ".join(str(c) for c in df.iloc[0].tolist() if str(c).strip())

        # Infer table topic
        content_str = df.to_string().lower()
        topic = infer_table_topic(content_str, table.page)

        # Create searchable text content
        text_content = df.to_string(index=False)

        # LangChain document for table
        table_doc = Document(
            page_content=(
                f"TABLE {i+1} (Page {table.page}) - {topic}\n"
                f"Columns: {col_names}\n\n"
                f"{text_content}"
            ),
            metadata={
                "source": "table",
                "table_id": i,
                "page": table.page,
                "topic": topic
            }
        )
        table_documents.append(table_doc)

        # Store table data
        extracted.append({
            "table_id": f"TABLE_{i+1}",
            "page": table.page,
            "df": df,
            "columns": df.iloc[0].tolist(),
            "topic": topic,
            "document": table_doc
        })

    return extracted, table_documents

def infer_table_topic(content_str, page_num):
    """Infer table topic from content"""
    topics = {
        "Based on your PDF"
    }

    for topic, keywords in topics.items():
        if any(kw in content_str for kw in keywords):
            return topic

    return f" Data (Page {page_num})"

# ============================================================
# BLIP-2 IMAGE CAPTIONING
# ============================================================
def load_blip2():
    """Load BLIP-2 model"""
    print("⏳ Loading BLIP-2...")
    processor = Blip2Processor.from_pretrained(BLIP2_MODEL)
    model = Blip2ForConditionalGeneration.from_pretrained(
        BLIP2_MODEL,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    print("✅ BLIP-2 loaded")
    return processor, model

def resize_img(img, max_side):
    """Resize image maintaining aspect ratio"""
    w, h = img.size
    if max(w, h) <= max_side:
        return img
    r = max_side / max(w, h)
    return img.resize((int(w * r), int(h * r)), Image.LANCZOS)

def parse_page_num(name):
    """Extract page number from filename"""
    m = PAGE_NUM_PATTERN.search(name)
    return int(m.group(1)) if m else None

def get_page_context(page_num, page_text_map, window=0):
    """Get text context from page"""
    if not page_num or page_num not in page_text_map:
        return ""

    text = page_text_map[page_num]
    if len(text) > 600:
        text = text[:600] + "..."

    return f"Page {page_num} context: {text}"

def infer_diagram_content(filename, page_num):
    """Infer diagram type from filename"""
    filename_lower = filename.lower()

    keywords_map = {
        'p77': "Discription of your own",
    }

    for keyword, description in keywords_map.items():
        if keyword in filename_lower:
            return f"{description} (Page {page_num})"

    return f"Technical diagram from  electrical installation (Page {page_num})"

def caption_all_images(folder, processor, model, page_text_map):
    """Caption all images and create LangChain documents"""
    prompt = "Question: What does this electrical diagram show? Answer:"
    device = next(model.parameters()).device
    image_documents = []
    image_data = []

    paths = sorted(p for p in folder.iterdir() if p.suffix.lower() in IMG_EXTS)
    print(f"📸 Captioning {len(paths)} images...")

    for idx, p in enumerate(paths, 1):
        try:
            page = parse_page_num(p.name)
            img = resize_img(Image.open(p).convert("RGB"), MAX_IMG_SIZE)

            inputs = processor(img, prompt, return_tensors="pt").to(device)

            with torch.no_grad():
                out = model.generate(**inputs, max_new_tokens=80)

            caption = processor.decode(out[0], skip_special_tokens=True)

            # Get context
            page_ctx = get_page_context(page, page_text_map, window=0)
            diagram_info = infer_diagram_content(p.name, page)

            # Create LangChain document
            image_doc = Document(
                page_content=(
                    f"Image: {p.name}\n"
                    f"Page: {page}\n"
                    f"Type: {diagram_info}\n"
                    f"Visual description: {caption}\n"
                    f"{page_ctx}"
                ),
                metadata={
                    "source": "image",
                    "filename": p.name,
                    "page": page,
                    "path": str(p)
                }
            )

            image_documents.append(image_doc)
            image_data.append({
                "document": image_doc,
                "path": p,
                "page": page,
                "filename": p.name,
                "caption": caption
            })

            if idx % 10 == 0:
                print(f"  Processed {idx}/{len(paths)}")

        except Exception as e:
            print(f"⚠️ Error processing {p.name}: {e}")

    print(f"✅ Captioned {len(image_data)} images")
    return image_data, image_documents

# ============================================================
# BUILD VECTOR STORE WITH LANGCHAIN
# ============================================================
def build_vectorstore(text_docs, table_docs, image_docs):
    """Build Chroma vector store with all documents"""
    print("🔍 Building vector store with LangChain...")

    # Combine all documents
    all_documents = text_docs + table_docs + image_docs

    print(f"   Total documents: {len(all_documents)}")
    print(f"   - Text chunks: {len(text_docs)}")
    print(f"   - Tables: {len(table_docs)}")
    print(f"   - Images: {len(image_docs)}")

    # Create embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    # Create Chroma vector store
    vectorstore = Chroma.from_documents(
        documents=all_documents,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )

    print("✅ Vector store built")
    return vectorstore

# ============================================================
# LOAD LLM
# ============================================================
def load_llm():
    """Load Mistral-7B with 4-bit quantization"""
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    print("⏳ Loading Mistral-7B in 4-bit...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=True
    )

    print("✅ Mistral-7B loaded")
    print(f"   GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

    return model, tokenizer

# ============================================================
# INITIALIZATION SEQUENCE
# ============================================================


# Step 1: Extract text and tables
print("\n📄 Step 1: Extracting text and tables...")
text_documents, page_text_map = extract_text_with_langchain(PDF_PATH)
tables_data, table_documents = extract_tables(PDF_PATH)
print(f"   ✓ {len(text_documents)} text chunks")
print(f"   ✓ {len(tables_data)} tables")

# Step 2: BLIP-2 image captioning
print("\n🖼️ Step 2: Captioning images with BLIP-2...")
blip_proc, blip_model = load_blip2()
images_data, image_documents = caption_all_images(
    IMAGES_FOLDER, blip_proc, blip_model, page_text_map
)

# Step 3: Free BLIP-2
print("\n🗑️ Step 3: Freeing BLIP-2...")
del blip_proc, blip_model
gc.collect()
torch.cuda.empty_cache()
print(f"   GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

# Step 4: Build vector store
print("\n🔍 Step 4: Building vector store...")
vectorstore = build_vectorstore(text_documents, table_documents, image_documents)

# Step 5: Load LLM
print("\n🤖 Step 5: Loading Mistral-7B...")
llm_model, tokenizer = load_llm()

print("\n" + "=" * 80)
print("✅ ALL SYSTEMS READY!")
print("=" * 80 + "\n")

# ============================================================
# CHATBOT FUNCTION - FIXED FOR NA VALUES
# ============================================================
def chatbot(query, history=None):
    """
    Main chatbot function using LangChain retrieval
    Returns: (markdown_text, list_of_images)
    """
    if not query.strip():
        return "Please enter a question.", []

    # ══════════════════════════════════════════════════════
    # RETRIEVE DOCUMENTS WITH LANGCHAIN
    # ══════════════════════════════════════════════════════

    # Retrieve top k documents
    retrieved_docs = vectorstore.similarity_search(query, k=12)

    # Separate by source type
    text_docs = []
    table_docs = []
    image_docs = []

    for doc in retrieved_docs:
        source = doc.metadata.get("source")
        if source == "pdf":
            text_docs.append(doc)
        elif source == "table":
            table_docs.append(doc)
        elif source == "image":
            image_docs.append(doc)

    # ══════════════════════════════════════════════════════
    # BUILD CONTEXT FROM RETRIEVED DOCUMENTS
    # ══════════════════════════════════════════════════════

    # Text context - take top 4 chunks
    text_ctx_parts = []
    for doc in text_docs[:4]:
        content = doc.page_content
        # Remove page markers for cleaner context
        content = re.sub(r'--- Page \d+ ---', '', content).strip()
        if len(content) > 700:
            content = content[:700] + "..."
        text_ctx_parts.append(content)

    text_ctx = "\n\n".join(text_ctx_parts)

    # Table context - find the actual table data
    table_md = ""
    table_ctx = ""
    displayed_tables = []

    for doc in table_docs[:3]:  # Top 3 tables
        table_id = doc.metadata.get("table_id")

        # Find the corresponding table data
        for t_data in tables_data:
            if t_data["table_id"] == f"TABLE_{table_id+1}":
                df_display = t_data["df"].copy()

                # CRITICAL FIX: fillna before any operations
                df_display = df_display.fillna('')

                # Use first row as headers
                df_display.columns = df_display.iloc[0]
                df_display = df_display[1:].reset_index(drop=True)

                # Ensure no NA values remain
                df_display = df_display.fillna('')

                # Markdown for display - now safe from NA errors
                try:
                    table_md += (
                        f"\n### 📊 {t_data['table_id']} - Page {t_data['page']} - {t_data['topic']}\n\n"
                        + df_display.to_markdown(index=False)
                        + "\n"
                    )
                except Exception as e:
                    # Fallback to HTML table if markdown fails
                    table_md += (
                        f"\n### 📊 {t_data['table_id']} - Page {t_data['page']} - {t_data['topic']}\n\n"
                        + df_display.to_html(index=False)
                        + "\n"
                    )

                # Context for LLM (first 10 rows only to save tokens)
                table_ctx += (
                    f"[Table {t_data['table_id']} on page {t_data['page']} - {t_data['topic']}]\n"
                    + df_display.head(10).to_string(index=False)
                    + "\n\n"
                )

                displayed_tables.append(t_data['table_id'])
                break

    # Image context
    relevant_images = []
    img_ctx = ""

    for doc in image_docs[:5]:  # Top 5 images
        filename = doc.metadata.get("filename")
        page = doc.metadata.get("page")
        img_path = doc.metadata.get("path")

        # Find the actual image data
        for img_data in images_data:
            if img_data["filename"] == filename:
                # For Gallery - return PIL Image object
                try:
                    pil_img = Image.open(img_path)
                    label = f"📄 Page {page} | {filename}"
                    relevant_images.append((pil_img, label))
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")

                # For LLM context
                img_ctx += (
                    f"[Image: {filename} from Page {page}]\n"
                    f"Description: {img_data['caption']}\n\n"
                )
                break

    # ══════════════════════════════════════════════════════
    # BUILD LLM PROMPT
    # ══════════════════════════════════════════════════════

    context_parts = []

    if text_ctx:
        context_parts.append(f"📄 DOCUMENT TEXT:\n{text_ctx}")

    if table_ctx:
        context_parts.append(f"📊 RELEVANT TABLES:\n{table_ctx}")

    if img_ctx:
        context_parts.append(f"🖼️ RELEVANT DIAGRAMS:\n{img_ctx}")

    if not context_parts:
        return "❌ No relevant information found in the document.", []

    combined_context = "\n\n".join(context_parts)

    prompt = f"""<s>[INST] You are a specialized technical assistant for the manual.

Use the provided context to answer the user's question accurately and technically.

CONTEXT:
{combined_context}

USER QUESTION: {query}



[/INST]

Answer:"""

    # ══════════════════════════════════════════════════════
    # GENERATE ANSWER
    # ══════════════════════════════════════════════════════

    # Truncate if needed
    tokens = tokenizer.encode(prompt, truncation=False)
    if len(tokens) > 2900:
        tokens = tokens[:2900]
        prompt = tokenizer.decode(tokens, skip_special_tokens=True)
        if "[/INST]" not in prompt[-100:]:
            prompt += "\n[/INST]\n\nAnswer:"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=3000
    ).to(llm_model.device)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            repetition_penalty=1.15,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract answer
    if "[/INST]" in full_response:
        answer = full_response.split("[/INST]")[-1].strip()
    else:
        answer = full_response.strip()

    if answer.startswith("Answer:"):
        answer = answer[7:].strip()

    # Clean up any context repetition
    # if len(answer) > 1500:
    #     answer = answer[:1500] + "..."

    # ══════════════════════════════════════════════════════
    # FORMAT OUTPUT
    # ══════════════════════════════════════════════════════

    output_md = f"### 🤖 Answer\n\n{answer}\n"

    # Add source pages
    source_pages = set()
    for doc in text_docs[:4]:
        if doc.metadata.get("page"):
            source_pages.add(doc.metadata["page"])

    if source_pages:
        pages_str = ", ".join([f"Page {p}" for p in sorted(source_pages)])
        output_md += f"\n*📄 Sources: {pages_str}*\n"

    # Add tables
    if table_md:
        output_md += f"\n---\n{table_md}"
        if displayed_tables:
            output_md += f"\n*📊 Displayed: {', '.join(displayed_tables)}*\n"

    # Note about images
    if relevant_images:
        output_md += f"\n---\n\n📎 **{len(relevant_images)} relevant diagram(s) shown below**\n"

    return output_md, relevant_images

# ============================================================
# GRADIO INTERFACE
# ============================================================

custom_css = """
#chatbot {
    height: 550px;
}
#gallery {
    min-height: 450px;
}
"""

with gr.Blocks(
    title="Technical Assistant",
    css=custom_css,
    theme=gr.themes.Soft()
) as demo:

    gr.Markdown(
        """
        # Chatbot

        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            chatbot_display = gr.Chatbot(
                label="Conversation",
                elem_id="chatbot",
                height=550,
                avatar_images=(None, "🤖")
            )

            with gr.Row():
                msg_input = gr.Textbox(
                    label="Your Question",
                    placeholder="Example: What does Table 1 show about generator routes?",
                    lines=3,
                    scale=4
                )

            with gr.Row():
                submit_btn = gr.Button("🚀 Send", variant="primary", scale=2)
                clear_btn = gr.Button("🗑️ Clear", scale=1)

        with gr.Column(scale=1):
            gr.Markdown("### 📊 Technical Diagrams")

            image_gallery = gr.Gallery(
                label="Relevant Diagrams",
                elem_id="gallery",
                columns=2,
                rows=2,
                height=450,
                object_fit="contain",
                show_label=False
            )


    # Event handlers
    def respond(message, chat_history):
        if not message.strip():
            return chat_history, []

        bot_response, images = chatbot(message)
        chat_history.append((message, bot_response))

        return chat_history, images

    def clear_all():
        return [], []

    submit_btn.click(
        fn=respond,
        inputs=[msg_input, chatbot_display],
        outputs=[chatbot_display, image_gallery]
    ).then(
        lambda: "",
        outputs=[msg_input]
    )

    msg_input.submit(
        fn=respond,
        inputs=[msg_input, chatbot_display],
        outputs=[chatbot_display, image_gallery]
    ).then(
        lambda: "",
        outputs=[msg_input]
    )

    clear_btn.click(
        fn=clear_all,
        outputs=[chatbot_display, image_gallery]
    )

print("\n🚀 Launching Gradio interface...")
demo.launch(share=True, server_port=7860, debug=True)